In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

from IPython.display import display  # noqa: F401

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Spin alignment

In [ ]:
import logging

import graphviz
import qrules
import sympy as sp
from IPython.display import Math

import ampform

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.ERROR)

As described in [[TR-015] Spin alignment](https://compwa-org--119.org.readthedocs.build/report/015.html), the {doc}`'standard' helicity formalism </usage/helicity/formalism>` is not suited for state transitions that have different decay topologies. For this reason, the {class}`.HelicityAmplitudeBuilder` insert a number of Wigner-$D$ function into the amplitude model in case there is more than one underlying {class}`~qrules.topology.Topology`. It is easiest to see this by inspecting the resulting {attr}`.HelicityModel.intensity` and its {attr}`~.HelicityModel.amplitudes`:

In [ ]:
reaction = qrules.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["K0", "Sigma+", "p~"],
    allowed_intermediate_particles=["Sigma(1660)", "N(1650)"],
    allowed_interaction_types=["strong"],
    formalism="helicity",
)

In [ ]:
dot = qrules.io.asdot(reaction, collapse_graphs=True, size=4)
graphviz.Source(dot)

In [ ]:
builder = ampform.get_builder(reaction)
model = builder.formulate()
model.intensity

In [ ]:
latex = sp.multiline_latex(
    sp.Symbol("I"),
    model.intensity.evaluate(),
    environment="eqnarray",
)
Math(latex)

This also introduces several **Wigner rotation angles** to the {attr}`.HelicityModel.kinematic_variables`:

In [ ]:
alpha = sp.Symbol("alpha_0^01", real=True)
model.kinematic_variables[alpha]

For more information about these angles, see [Compute Wigner rotation angles](https://compwa-org--119.org.readthedocs.build/report/015.html#compute-wigner-rotation-angles) in TR-015.

Spin alignment can be switched off or on by setting {attr}`.HelicityAmplitudeBuilder.align_spin`:

In [ ]:
builder.align_spin = False
non_aligned_model = builder.formulate()
non_aligned_model.intensity